In [ ]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to change the path if needed.)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read the School Data and Student Data and store into a Pandas DataFrame
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Cleaning Student Names and Replacing Substrings in a Python String
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

# Check names.
student_data_df.head(10)

## Deliverable 1: Replace the reading and math scores.

### Replace the 9th grade reading and math scores at Thomas High School with NaN.

In [ ]:
# Install numpy using conda install numpy or pip install numpy. 
# Step 1. Import numpy as np.
import numpy as np

In [ ]:
# Step 2. Use the loc method on the student_data_df to select all the reading scores from the 9th grade at Thomas High School and replace them with NaN.
student_data_df.loc[(student_data_df["grade"] == "9th") & (student_data_df["school_name"] == "Thomas High School"), "reading_score"] = np.nan
student_data_df

In [ ]:
#  Step 3. Refactor the code in Step 2 to replace the math scores with NaN.
student_data_df.loc[(student_data_df["grade"] == "9th") & (student_data_df["school_name"] == "Thomas High School"), "math_score"] = np.nan

In [ ]:
#  Step 4. Check the student data for NaN's. 
student_data_df.tail(10)

## Deliverable 2 : Repeat the school district analysis

### District Summary

In [ ]:
# Combine the data into a single dataset
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
school_data_complete_df.tail(7)

In [ ]:
# Calculate the Totals (Schools and Students)
school_count = len(school_data_complete_df["school_name"].unique())
student_count = school_data_complete_df["Student ID"].count()

# Calculate the Total Budget
total_budget = school_data_df["budget"].sum()

In [ ]:
# Calculate the Average Scores using the "clean_student_data".
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

In [ ]:
# Step 1. Get the number of students that are in ninth grade at Thomas High School.
# These students have no grades. 
##thomas_9th_math_count = school_data_complete_df["math_score"].isnull().sum()
thomas_9th_grade_math_count = school_data_complete_df.loc[(school_data_complete_df["grade"] == "9th") & (school_data_complete_df["school_name"] == "Thomas High School"), "Student ID"].count()

# Get the total student count 
student_count = school_data_complete_df["Student ID"].count()

# Step 2. Subtract the number of students that are in ninth grade at 
# Thomas High School from the total student count to get the new total student count.
student_count_new = (student_count - thomas_9th_math_count)
student_count_new

In [ ]:
# Calculate the passing rates using the "clean_student_data".
passing_math_count = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]

In [ ]:
# Step 3. Calculate the passing percentages with the new total student count.
passing_math_percentage = ((passing_math_count / student_count_new) * 100)
passing_reading_percentage = ((passing_reading_count / student_count_new) * 100)

In [ ]:
# Calculate the students who passed both reading and math.
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)
                                               & (school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students that passed both reading and math.
overall_passing_math_reading_count = passing_math_reading["student_name"].count()


# Step 4.Calculate the overall passing percentage with new total student count.
overall_passing_percentage = overall_passing_math_reading_count / student_count_new * 100
overall_passing_percentage


In [ ]:
# Create a DataFrame
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count, 
          "Total Students": student_count, 
          "Total Budget": total_budget,
          "Average Math Score": average_math_score, 
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])



# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# Format the "Total Budget" to have the comma for a thousands separator, a decimal separator and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)

# Display the data frame
district_summary_df

##  School Summary

In [ ]:
# Determine the School Type
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both reading and math.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                               & (school_data_complete_df["math_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [ ]:
# Create the DataFrame
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})


per_school_summary_df.tail()

In [ ]:
# Format the Total School Budget and the Per Student Budget
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
per_school_summary_df.head()

In [ ]:
# Step 5.  Get the number of 10th-12th graders from Thomas High School (THS).
thomas_10th_to_12th_all_count = school_data_complete_df.loc[(school_data_complete_df["grade"] != "9th") & (school_data_complete_df["school_name"] == "Thomas High School"), "Student ID"].count()
thomas_10th_to_12th_all_count

In [ ]:
# Step 6. Get all the students passing math from THS
thomas_all_passing_math = school_data_complete_df.loc[(school_data_complete_df["school_name"] == "Thomas High School") & (school_data_complete_df["math_score"] >= 70), "student_name"].count()
thomas_all_passing_math
#thomas_10th_to_12th_passing_math = school_data_complete_df.loc[(school_data_complete_df["grade"] != "9th") & (school_data_complete_df["school_name"] == "Thomas High School") & (school_data_complete_df["math_score"] >= 70), "Student ID"].count()
#thomas_10th_to_12th_passing_math

In [ ]:
# Step 7. Get all the students passing reading from THS
thomas_all_passing_reading = school_data_complete_df.loc[(school_data_complete_df["school_name"] == "Thomas High School") & (school_data_complete_df["reading_score"] >= 70), "student_name"].count()
thomas_all_passing_reading

In [ ]:
# Step 8. Get all the students passing math and reading from THS
thomas_10th_to_12th_passing_count = school_data_complete_df.loc[(school_data_complete_df["grade"] != "9th") & (school_data_complete_df["school_name"] == "Thomas High School") & (school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70), "Student ID"].count()
thomas_10th_to_12th_passing_count

In [ ]:
# Step 9. Calculate the percentage of 10th-12th grade students passing math from Thomas High School. 
thomas_10th_to_12th_passing_math_percentage = (thomas_all_passing_math / thomas_10th_to_12th_all_count) * 100
thomas_10th_to_12th_passing_math_percentage

In [ ]:
# Step 10. Calculate the percentage of 10th-12th grade students passing reading from Thomas High School.
thomas_10th_to_12th_passing_reading_percentage = (thomas_all_passing_reading / thomas_10th_to_12th_all_count) * 100
thomas_10th_to_12th_passing_reading_percentage

In [ ]:
# Step 11. Calculate the overall passing percentage of 10th-12th grade from Thomas High School. 
thomas_10th_to_12th_overall_passing_percentage = (thomas_10th_to_12th_passing_count / thomas_10th_to_12th_all_count) * 100
thomas_10th_to_12th_overall_passing_percentage

In [ ]:
# Step 12. Replace the passing math percent for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc[per_school_summary_df.index == "Thomas High School", ["% Passing Math"]] = thomas_10th_to_12th_passing_math_percentage
#per_school_summary_df.tail()

In [ ]:
# Step 13. Replace the passing reading percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc[per_school_summary_df.index == "Thomas High School", ["% Passing Reading"]] = thomas_10th_to_12th_passing_reading_percentage
#per_school_summary_df.tail()

In [ ]:
# Step 14. Replace the overall passing percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc[per_school_summary_df.index == "Thomas High School", ["% Overall Passing"]] = thomas_10th_to_12th_overall_passing_percentage
#per_school_summary_df.tail()

In [ ]:
# per_school_summary_df
per_school_summary_df

## High and Low Performing Schools 

In [ ]:
# Sort and show top five schools.
top_five_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)
top_five_schools.head(5)

In [ ]:
# Sort and show top five schools.
bottom_five_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=True)
bottom_five_schools.head(5)

## Math and Reading Scores by Grade

In [ ]:
# Create a Series of scores by grade levels using conditionals.
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

#ninth_grade_math_scores = school_data_complete_df[(school_data_complete_df["grade"] == "9th")].groupby(["school_name"]).mean()["math_score"]
#tenth_grade_math_scores = school_data_complete_df[(school_data_complete_df["grade"] == "10th")].groupby(["school_name"]).mean()["math_score"]
#eleventh_grade_math_scores = school_data_complete_df[(school_data_complete_df["grade"] == "11th")].groupby(["school_name"]).mean()["math_score"]
#twelfth_grade_math_scores = school_data_complete_df[(school_data_complete_df["grade"] == "12th")].groupby(["school_name"]).mean()["math_score"]

# Group each school Series by the school name for the average math score.
ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

# Group each school Series by the school name for the average reading score.
ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]
#ninth_grade_reading_scores

In [ ]:
# Combine each Series for average math scores by school into single data frame.
math_scores_by_grade = pd.DataFrame({
               "9th": ninth_grade_math_scores,
               "10th": tenth_grade_math_scores,
               "11th": eleventh_grade_math_scores,
               "12th": twelfth_grade_math_scores})
math_scores_by_grade.tail()

In [478]:
# Combine each Series for average reading scores by school into single data frame.
reading_scores_by_grade = pd.DataFrame({
              "9th": ninth_grade_reading_scores,
              "10th": tenth_grade_reading_scores,
              "11th": eleventh_grade_reading_scores,
              "12th": twelfth_grade_reading_scores})
reading_scores_by_grade.tail()

,9th,10th,11th,12th
school_name,,,,
Rodriguez High School,80.993127,80.629808,80.864811,80.376426
Shelton High School,84.122642,83.441964,84.373786,82.781671
Thomas High School,NaN,84.254157,83.585542,83.831361
Wilson High School,83.939778,84.021452,83.764608,84.317673
Wright High School,83.833333,83.812757,84.156322,84.073171


In [479]:
# Format each grade column.
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.1f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.1f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.1f}".format)

reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:,.1f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:,.1f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:,.1f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:,.1f}".format)
reading_scores_by_grade.head(2)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3


In [480]:
# Remove the index.
math_scores_by_grade.index.name = None

# Display the data frame
math_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4


In [481]:
## Remove the index.
reading_scores_by_grade.index.name = None

# Display the data frame
reading_scores_by_grade.tail()

,9th,10th,11th,12th
Rodriguez High School,81.0,80.6,80.9,80.4
Shelton High School,84.1,83.4,84.4,82.8
Thomas High School,nan,84.3,83.6,83.8
Wilson High School,83.9,84.0,83.8,84.3
Wright High School,83.8,83.8,84.2,84.1


## Scores by School Spending

In [482]:
# Establish the spending bins and group names.
per_school_capita.describe()
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

# Categorize spending based on the bins.
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)
per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-629
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$584
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-644
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-644
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-629


In [483]:
# Calculate averages for the desired columns. 
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]


In [484]:
# Create the DataFrame
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})
spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,81.899826,83.155286,87.133538,92.718205,81.418596
$630-644,78.502002,81.636261,73.462589,84.319261,62.778233
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


In [485]:
# Format the DataFrame 
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)
spending_summary_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,81.9,83.2,87,93,81
$630-644,78.5,81.6,73,84,63
$645-675,77.0,81.0,66,81,54


## Scores by School Size

In [486]:
# Establish the bins.
per_school_counts.describe
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Categorize spending based on the bins.
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)
per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-629,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$584,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-644,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-644,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-629,Medium (1000-2000)


In [487]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]


In [488]:
# Assemble into DataFrame. 
size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})
#size_summary_df

In [489]:
# Format the DataFrame  
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)
size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)
size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)
size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)
size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.9,94,96,90
Medium (1000-2000),83.361201,83.9,94,97,91
Large (2000-5000),77.746417,81.3,70,83,58


## Scores by School Type

In [496]:
# Calculate averages for the desired columns. 
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"] .map("{:.1f}".format)
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"].map("{:.1f}".format)
type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"].map("{:.0f}".format)
type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"].map("{:.0f}".format)
type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"].map("{:.0f}".format)

In [497]:
# Assemble into DataFrame. 
type_summary_df = pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})
type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54


In [499]:
# # Format the DataFrame 
# already formatted during series formation.
type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54


In [ ]:
# Add the Pandas dependency.
import pandas as pd

In [ ]:
# Files to load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [ ]:
# Read the school data file and store it in a Pandas DataFrame.
school_data_df = pd.read_csv(school_data_to_load)
school_data_df

In [ ]:
# Read the student data file and store it in a Pandas DataFrame.
student_data_df = pd.read_csv(student_data_to_load)
student_data_df.head(4)

In [ ]:
# Determine if there are any missing values in the school data.
school_data_df.count()

In [ ]:
# Determine if there are any missing values in the student data.
student_data_df.count()

In [ ]:
# Determine if there are any missing values in the school data.
school_data_df.isnull()

In [ ]:
# Determine if there are any missing values in the student data.
student_data_df.isnull()

In [ ]:
# Determine if there are any missing values in the student data.
student_data_df.isnull().sum()

In [ ]:
# Determine if there are any missing values in the school data.
school_data_df.notnull()

In [ ]:
# Determine if there are not any missing values in the student data.
student_data_df.notnull().sum()

In [ ]:
# Determine data types for the school DataFrame.
school_data_df.dtypes


In [ ]:
school_data_df.info

In [ ]:
# Find data type of budget column.
school_data_df["budget"].dtype

In [ ]:
school_data_df.budget.dtype

In [ ]:
# Determine data types for the student DataFrame.
student_data_df.dtypes

In [ ]:
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]
# Remember that there is a whitespace between the prefix and the first

In [ ]:
# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

In [ ]:
student_data_df.head(10)

In [ ]:
# Combine the data into a single dataset.
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name"])
school_data_complete_df.head(10)

In [ ]:
# Get the total number of students.
student_count = school_data_complete_df["Student ID"].count()
student_count

In [ ]:
# Get the total number of schools.
school_count = school_data_df["school_name"].count()
school_count

In [ ]:
school_count2 = school_data_complete_df["school_name"].unique()
school_count2

In [ ]:
school_count3 = len(school_data_complete_df["school_name"].unique())
school_count3

In [ ]:
# Calculate the total budget.
total_budget = school_data_df["budget"].sum()
total_budget

In [ ]:
total_budget1 = sum(school_data_complete_df["budget"].unique())
total_budget1

In [ ]:
# Calculate the average reading score.
average_reading_score = school_data_complete_df["reading_score"].mean()
average_reading_score

In [ ]:
# Calculate the average math score.
average_math_score = school_data_complete_df["math_score"].mean()
average_math_score

In [ ]:
# Get all the students who are passing math in a new DataFrame.
passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]
passing_math.head()

In [ ]:
# Get all the students that are passing reading in a new DataFrame.
passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]
passing_reading.head()

In [ ]:
# Calculate the number of students passing reading.
passing_reading_count = passing_reading["student_name"].count()
passing_reading_count

In [ ]:
# Calculate the number of students passing math.
passing_math_count = passing_math["student_name"].count()
passing_math_count

In [ ]:
len(passing_math)

In [ ]:
passing_math.count()

In [ ]:
# Calculate the percent that passed math.
passing_math_percentage = (passing_math_count / student_count) * 100
passing_math_percentage

In [ ]:
# Calculate the percent that passed reading.
passing_reading_percentage = (passing_reading_count / student_count) * 100
passing_reading_percentage

In [ ]:
# Calculate the students who passed both math and reading.
overall_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]
overall_passing_math_reading

In [ ]:
# Calculate the number of students who passed both math and reading.
overall_passing_math_reading_count = overall_passing_math_reading["student_name"].count()
overall_passing_math_reading_count

In [ ]:
overall_passing_math_reading_count1 = overall_passing_math_reading.student_name.count()
overall_passing_math_reading_count1

In [ ]:
# Calculate the overall passing percentage.
overall_passing_percentage = overall_passing_math_reading_count * 100 / student_count
overall_passing_percentage

In [ ]:
# Adding a list of values with keys to create a new DataFrame.
district_summary_df = pd.DataFrame(
    [{"Total Schools": school_count,
     "Total Students": student_count,
     "Total Budget": total_budget,
     "Average Math Score": average_math_score,
     "Average Reading Score": average_reading_score,
     "% Passing Math": passing_math_percentage,
     "% Passing Reading": passing_reading_percentage,
     "% Overall Passing": overall_passing_percentage}])
district_summary_df

In [ ]:
# Define the function "say_hello" so it prints "Hello!" when called.
def say_hello():
    print("Hi there!")

In [ ]:
say_hello()

In [ ]:
# Define a function that calculates the percentage of students that passed both # math and reading and prints the passing percentage to the output when the
# function is called.
def passing_math_percent(pass_math_count, student_count):
    return pass_math_count / float(student_count) * 100

In [ ]:
passing_math_count = 29370
total_student_count = 39170
# Call the function.
passing_math_percent(passing_math_count, total_student_count)

In [ ]:
# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Students"]

In [ ]:
# Format "Total Budget" to have the comma for a thousands separator, a decimal separator, and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Total Budget"]

In [ ]:
# Format the columns.
import pandas as pd
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
#district_summary_df["Average Math Score"]

district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
#district_summary_df["Average Reading Score"]

district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.0f}".format)
#district_summary_df["% Passing Reading"]

district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}".format)
#district_summary_df["% Passing Math"]

district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.0f}".format)
#district_summary_df["% Overall Passing"]

district_summary_df

In [ ]:
# Reorder the columns in the order you want them to appear.
new_column_order = ["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

# Assign district summary df the new column order.
district_summary_df = district_summary_df[new_column_order]
district_summary_df

In [ ]:
school_data_df

In [ ]:
# Determine the school type.
per_school_types = school_data_df.set_index(["school_name"])["type"]
per_school_types

In [ ]:
# Add the per_school_types into a DataFrame for testing.
df = pd.DataFrame(per_school_types)
df

In [ ]:
# Calculate the total student count.
per_school_counts = school_data_df["size"]
per_school_counts

In [ ]:
# Calculate the total student count.
per_school_counts = school_data_df.set_index(["school_name"])["size"]
per_school_counts

In [ ]:
# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()
per_school_counts

In [ ]:
# Calculate the total school budget.
per_school_budget = school_data_df.set_index(["school_name"])["budget"]
per_school_budget

In [ ]:
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts
per_school_capita

In [ ]:
school_data_complete_df.head()

In [ ]:
# Calculate the average math scores.
per_school_average = school_data_complete_df.groupby(["school_name"]).mean()
per_school_average

In [ ]:
# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_math

In [ ]:
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]
per_school_reading

In [ ]:
# To get the passing percentages, we need to:
 # 1. Determine what is the passing grade.
 # 2. Get the number of students who passed math and reading.
 # 3. Get the students who passed math and passed reading

In [ ]:
student_data_df.head(4)

In [ ]:
# Get the no. of students who passed Math and Reading
passing_math_reading = student_data_df[(student_data_df["math_score"] >= 70) & (student_data_df["reading_score"] >= 70)]
passing_math_reading

In [ ]:
# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_math

In [ ]:
# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]
per_school_passing_reading

In [ ]:
per_school_passing_math.set_index(["school_name"])

In [ ]:
per_school_passing_math.groupby(["school_name"])

In [ ]:
# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_math.head()

In [ ]:
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading.head()

In [ ]:
# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100

per_school_passing_reading = per_school_passing_reading / per_school_counts * 100
per_school_passing_math

In [ ]:
passing_math_reading

In [ ]:
# Calculate the students who passed both math and reading.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]

per_passing_math_reading.head()

In [ ]:
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]
per_passing_math_reading

In [ ]:
# Calculate the overall passing percentage.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100
per_overall_passing_percentage


In [ ]:
# Create a new dataframe "per_school_summary_df"
per_school_summary_df = pd.DataFrame(
    {"School Type": per_school_types,
     "Total Students": per_school_counts,
     "Total School Budget": per_school_budget,
     "Per Student Budget": per_school_capita,
     "Average Math Score": per_school_math,
     "Average Reading Score": per_school_reading,
     "% Passing Math": per_school_passing_math,
     "% Passing Reading": per_school_passing_reading,
     "% Overall Passing": per_overall_passing_percentage})
per_school_summary_df.head()

In [ ]:
# Format the Total School Budget and the Per Student Budget columns.
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)
per_school_summary_df

In [ ]:
# Sort and show top 5 schools.
top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)
top_schools.head()

In [ ]:
# Sort and show bottom five schools.
bottom_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=True)
bottom_schools.head()

In [ ]:
school_data_complete_df.head()

In [ ]:
# Create 9th grader Series.
ninth_graders = school_data_complete_df[school_data_complete_df["grade"] == "9th"]
#ninth_graders
#Create a grade level DataFrames.
tenth_graders = school_data_complete_df[school_data_complete_df["grade"] == "10th"]
eleventh_graders = school_data_complete_df[school_data_complete_df["grade"] == "11th"]
twelfth_graders = school_data_complete_df[school_data_complete_df["grade"] == "12th"]
ninth_graders.head()

In [ ]:
# Group each school Series by the school name for the average math score.
ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"].map("{:.1f}".format)
ninth_grade_math_scores

In [ ]:
tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"].map("{:.1f}".format)
eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"].map("{:.1f}".format)
twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"].map("{:.1f}".format)
eleventh_grade_math_scores

In [ ]:
# Group each school Series by the school name for the average reading score.
ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"].map("{:.1f}".format)
ninth_grade_reading_scores

In [ ]:
tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"].map("{:.1f}".format)
eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"].map("{:.1f}".format)
twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"].map("{:.1f}".format)

In [ ]:
twelfth_grade_reading_scores

In [ ]:
# Combine each Series for average math scores by school into single DataFrame.
math_scores_by_grade = pd.DataFrame(
    {"9th": ninth_grade_math_scores,
    "10th": tenth_grade_math_scores,
    "11th": eleventh_grade_math_scores,
    "12th": twelfth_grade_math_scores})
math_scores_by_grade.head()

In [ ]:
# Combine each Series for average reading scores by school into single DataFrame.
reading_scores_by_grade = pd.DataFrame(
    {"9th": ninth_grade_reading_scores,
    "10th": tenth_grade_reading_scores,
    "11th": eleventh_grade_reading_scores,
    "12th": twelfth_grade_reading_scores})
reading_scores_by_grade.head()

In [ ]:
# Remove the index name in math grade df.
math_scores_by_grade.index.name = None
math_scores_by_grade.head()

In [ ]:
# Remove the index name in reading grade df.
reading_scores_by_grade.index.name = None
reading_scores_by_grade.head()

In [ ]:
per_school_capita.head()


In [ ]:
per_school_capita.sort_values(ascending=False)

In [ ]:
# Get the descriptive statistics for the per_school_capita.
per_school_capita.describe()

In [ ]:
# Cut the per_school_capita into the spending ranges.
spending_bins = [0, 585, 615, 645, 675]
pd.cut(per_school_capita, spending_bins)

In [ ]:
# Get the number of schools in each range or bin
spending_bins = [0, 585, 615, 645, 675]
per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()

In [ ]:
# Establish the spending bins and group names.
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

In [ ]:
# Categorize spending based on the bins.
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)
per_school_summary_df

In [ ]:
# Calculate averages for the desired columns.
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_math_scores

In [ ]:
spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]
overall_passing_spending

In [ ]:
# Assemble into DataFrame.
spending_summary_df = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})
spending_summary_df

In [ ]:
# Formatting spending_summary_df
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)
spending_summary_df

In [ ]:
per_school_summary_df

In [ ]:
per_school_counts

In [ ]:
per_school_counts.describe()

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)
per_school_summary_df.head()

In [ ]:
per_school_summary_df.drop(columns=["Total Student"])

In [ ]:
# Calculate averages for the desired columns.
size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_math_scores

In [ ]:
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into dataframe.
size_summary_df = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
})
size_summary_df

In [ ]:
# Formatting.
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)
size_summary_df

In [ ]:
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)

size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)

size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)

size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)

size_summary_df

In [ ]:
# Calculate averages for desired columns.
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"].map("{:.1f}".format)
type_math_scores

In [ ]:
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"].map("{:.1f}".format)
type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"].map("{:.0f}".format)
type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"].map("{:.0f}".format)
type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"].map("{:.0f}".format)
type_overall_passing

In [ ]:
# Assemble into DataFrame school_type_df.
type_summary_df = pd.DataFrame({
    "Average Math Score": type_math_scores,
    "Average Reading Score": type_reading_scores,
    "% Passing Math": type_passing_math,
    "% Passing Reading": type_passing_reading,
    "% Overall Passing": type_overall_passing
})
type_summary_df